In [ ]:
%load_ext jupyter_black

In [ ]:
import pathlib as pl

import hvplot.xarray  # noqa
import pywatershed as pws
import xarray as xr

repo_root = pws.constants.__pywatershed_root__.parent

In [ ]:
domain_name = "drb_2yr"
domain_dir = repo_root / f"test_data/{domain_name}"
input_dir = domain_dir / "output"
output_dir = pl.Path("./prms_snow17")
output_dir.mkdir(exist_ok=True)

## control and parameters

In [ ]:
ctl_prms = pws.Control.load_prms(
    domain_dir / "control.test", warn_unused_options=False
)
output_dir_pws = output_dir / "output_pws"
ctl_prms.options = ctl_prms.options | {
    "input_dir": input_dir,
    "netcdf_output_dir": output_dir_pws,
    "netcdf_output_var_names": pws.PRMSSnow.get_variables(),
}
dis_prms = pws.Parameters.from_netcdf(
    domain_dir / "parameters_dis_hru.nc", encoding=False
)
params_prms = pws.parameters.PrmsParameters.load(domain_dir / "myparam.param")

input_vars_prms = {}
for key in pws.PRMSSnow.get_inputs():
    nc_path = input_dir / f"{key}.nc"
    input_vars_prms[key] = nc_path

In [ ]:
output_dir_s17 = output_dir / "output_snow17"

ctl_s17 = pws.Control(
    start_time=ctl_prms.start_time,
    end_time=ctl_prms.end_time,
    time_step=ctl_prms.time_step,
    options={
        "input_dir": input_dir,
        "netcdf_output_dir": output_dir_s17,
        "netcdf_output_var_names": pws.Snow17.get_variables(),
    },
)
param_file_s17 = domain_dir / "drb_snow17_params_default.nc"
params_s17 = pws.Parameters.from_netcdf(param_file_s17)
# for now, just duplicate the parameters into discretization
dis_s17 = pws.Parameters.from_netcdf(param_file_s17)
input_vars_s17 = {}
for key in pws.Snow17.get_inputs():
    nc_path = input_dir / f"{key}.nc"
    input_vars_s17[key] = nc_path

## prms

In [ ]:
prms_snow = pws.PRMSSnow(
    ctl_prms, dis_prms, params_prms, **input_vars_prms, budget_type="error"
)
prms_snow.initialize_netcdf()
for istep in range(ctl_prms.n_times):
    ctl_prms.advance()
    prms_snow.advance()
    prms_snow.calculate(1.0)
    prms_snow.output()

prms_snow.finalize()

In [ ]:
prms_snow.budget

In [ ]:
## snow-17

In [ ]:
# The budget, as expressed, fails to close on many occasions.
# Is the budget wrong somehow? Not obvious how. Can probe the conditions later.
snow17 = pws.Snow17(
    ctl_s17, dis_s17, params_s17, **input_vars_s17, budget_type="warn"
)
snow17.initialize_netcdf()
for istep in range(ctl_s17.n_times):
    ctl_s17.advance()
    snow17.advance()
    snow17.calculate(1.0)
    snow17.output()

snow17.finalize()

In [ ]:
snow17.budget

## Compare snow variables

In [ ]:
def plot_compare_var(var_name):
    ds_pws = (
        (
            xr.open_dataarray(output_dir_pws / f"{var_name}.nc")
            .rename("pywatershed")
            .to_dataset()
        )
        .drop("nhm_id")
        .rename(nhm_id="hru_id")
    )
    ds_s17 = (
        xr.open_dataarray(output_dir_s17 / f"{var_name}.nc")
        .rename("snow-17")
        .to_dataset()
    )
    ds_pws["hru_id"] = ds_s17["hru_id"]
    ds_plot = xr.merge([ds_pws, ds_s17])
    display(ds_plot.hvplot(x="time", groupby="hru_id", title=var_name))
    return

In [ ]:
plot_compare_var(pws.Snow17.get_variables()[0])

In [ ]:
plot_compare_var(pws.Snow17.get_variables()[1])

In [ ]:
plot_compare_var(pws.Snow17.get_variables()[2])

In [ ]:
plot_compare_var(pws.Snow17.get_variables()[3])

In [ ]:
plot_compare_var(pws.Snow17.get_variables()[4])

In [ ]:
plot_compare_var(pws.Snow17.get_variables()[5])

In [ ]:
plot_compare_var(pws.Snow17.get_variables()[6])

In [ ]:
plot_compare_var(pws.Snow17.get_variables()[7])

In [ ]:
plot_compare_var(pws.Snow17.get_variables()[8])